## 由于春节时间较紧，直接爬了一个有距离的网站，可能与高德地图的数据不同，之后再进行优化

In [99]:
import requests
from bs4 import BeautifulSoup
import re
import os
 
ul = 'https://www.bjsubway.com/station/zjgls/#'
response = requests.get(ul)
response.encoding = 'gbk'  # 原始网页编码错误，utf-8也不管用，只能用gbk
html = response.text
# print(html)
soup = BeautifulSoup(html, 'lxml')  # 网页解析
 
 
def get_txt_name():  # 得到线路名称的前一步
    txt_src_name = []
    for i in range(5, 10):
        temp = soup.find_all('td', {'colspan': str(i)})
        txt_src_name += temp
    return txt_src_name
    # 格式如[<td colspan="6">15号线相邻站间距信息统计表</td>, <td colspan="6">昌平线相邻站间距信息统计表</td>]
# print(get_txt_name())  # 测试用
def get_txtuseful_name():  # 得到可用的线路名称
    obj = []
    for each in get_txt_name():
        temp = re.findall(r">(.+?)<", str(each))  # 从>匹配到<(不包含)，若要包含，则先使用re.compile，再search
        obj += temp
    return obj
# print(get_txtuseful_name())  # 测试用
 
Stationinfo = soup.find_all('tbody')
def get_stationinfo():
    obj = []
    for each in Stationinfo:
        temp = re.findall(r">(.+?)<", str(each))
        obj += temp
    return obj
# print(get_stationinfo())
 
 
station_list = get_stationinfo()
# print(station_list)

In [100]:
station_list

['苹果园——古城',
 '2606',
 '上行/下行',
 '古城——八角游乐园',
 '1921',
 '上行/下行',
 '八角游乐园——八宝山',
 '1953',
 '上行/下行',
 '八宝山——玉泉路',
 '1479',
 '上行/下行',
 '玉泉路——五棵松',
 '1810',
 '上行/下行',
 '五棵松——万寿路',
 '1778',
 '上行/下行',
 '万寿路——公主坟',
 '1313',
 '上行/下行',
 '公主坟——军事博物馆',
 '1172',
 '上行/下行',
 '军事博物馆——木樨地',
 '1166',
 '上行/下行',
 '木樨地——南礼士路',
 '1291',
 '上行/下行',
 '南礼士路——复兴门',
 '424',
 '上行/下行',
 '复兴门——西单',
 '1590',
 '上行/下行',
 '西单——天安门西',
 '1217',
 '上行/下行',
 '天安门西——天安门东',
 '925',
 '上行/下行',
 '天安门东——王府井',
 '852',
 '上行/下行',
 '王府井——东单',
 '774',
 '上行/下行',
 '东单——建国门',
 '1230',
 '上行/下行',
 '建国门——永安里',
 '1377',
 '上行/下行',
 '永安里——国贸',
 '790',
 '上行/下行',
 '国贸——大望路',
 '1385',
 '上行/下行',
 '大望路——四惠',
 '1673',
 '上行/下行',
 '四惠——四惠东',
 '1714',
 '上行/下行',
 '西直门——车公庄',
 '909',
 '上行/下行',
 '车公庄——阜成门',
 '960',
 '上行/下行',
 '阜成门——复兴门',
 '1832',
 '上行/下行',
 '复兴门——长椿街',
 '1234',
 '上行/下行',
 '长椿街——宣武门',
 '929',
 '上行/下行',
 '宣武门——和平门',
 '851',
 '上行/下行',
 '和平门——前门',
 '1171',
 '上行/下行',
 '前门——崇文门',
 '1634',
 '上行/下行',
 '崇文门——北京站',
 '1023',
 '上行/下行',
 '北京站——建国门',
 '

In [101]:
from collections import defaultdict

def build_connection(station_list):
    station_connection = defaultdict(list)
    for i in station_list:
        if re.search('——', i):   #匹配地铁路线的上下行车站
            station_connection[i.split('——')[0]].append(i.split('——')[1])
            station_connection[i.split('——')[1]].append(i.split('——')[0])
            #容易遗漏上行路线
        else: continue
    return station_connection

In [102]:
station_connection=build_connection(station_list)

In [103]:
station_connection

defaultdict(list,
            {'苹果园': ['古城'],
             '古城': ['苹果园', '八角游乐园'],
             '八角游乐园': ['古城', '八宝山'],
             '八宝山': ['八角游乐园', '玉泉路'],
             '玉泉路': ['八宝山', '五棵松'],
             '五棵松': ['玉泉路', '万寿路'],
             '万寿路': ['五棵松', '公主坟'],
             '公主坟': ['万寿路', '军事博物馆', '莲花桥', '西钓鱼台'],
             '军事博物馆': ['公主坟', '木樨地', '白堆子', '北京西站'],
             '木樨地': ['军事博物馆', '南礼士路'],
             '南礼士路': ['木樨地', '复兴门'],
             '复兴门': ['南礼士路', '西单', '阜成门', '长椿街'],
             '西单': ['复兴门', '天安门西', '灵境胡同', '宣武门'],
             '天安门西': ['西单', '天安门东'],
             '天安门东': ['天安门西', '王府井'],
             '王府井': ['天安门东', '东单'],
             '东单': ['王府井', '建国门', '灯市口', '崇文门'],
             '建国门': ['东单', '永安里', '北京站', '朝阳门'],
             '永安里': ['建国门', '国贸'],
             '国贸': ['永安里', '大望路', '金台夕照', '双井'],
             '大望路': ['国贸', '四惠', '九龙山', '红庙'],
             '四惠': ['大望路', '四惠东', '四惠东'],
             '四惠东': ['四惠', '四惠', '高碑店'],
             '西直门': ['车公庄', 

In [105]:
station_list

['苹果园——古城',
 '2606',
 '上行/下行',
 '古城——八角游乐园',
 '1921',
 '上行/下行',
 '八角游乐园——八宝山',
 '1953',
 '上行/下行',
 '八宝山——玉泉路',
 '1479',
 '上行/下行',
 '玉泉路——五棵松',
 '1810',
 '上行/下行',
 '五棵松——万寿路',
 '1778',
 '上行/下行',
 '万寿路——公主坟',
 '1313',
 '上行/下行',
 '公主坟——军事博物馆',
 '1172',
 '上行/下行',
 '军事博物馆——木樨地',
 '1166',
 '上行/下行',
 '木樨地——南礼士路',
 '1291',
 '上行/下行',
 '南礼士路——复兴门',
 '424',
 '上行/下行',
 '复兴门——西单',
 '1590',
 '上行/下行',
 '西单——天安门西',
 '1217',
 '上行/下行',
 '天安门西——天安门东',
 '925',
 '上行/下行',
 '天安门东——王府井',
 '852',
 '上行/下行',
 '王府井——东单',
 '774',
 '上行/下行',
 '东单——建国门',
 '1230',
 '上行/下行',
 '建国门——永安里',
 '1377',
 '上行/下行',
 '永安里——国贸',
 '790',
 '上行/下行',
 '国贸——大望路',
 '1385',
 '上行/下行',
 '大望路——四惠',
 '1673',
 '上行/下行',
 '四惠——四惠东',
 '1714',
 '上行/下行',
 '西直门——车公庄',
 '909',
 '上行/下行',
 '车公庄——阜成门',
 '960',
 '上行/下行',
 '阜成门——复兴门',
 '1832',
 '上行/下行',
 '复兴门——长椿街',
 '1234',
 '上行/下行',
 '长椿街——宣武门',
 '929',
 '上行/下行',
 '宣武门——和平门',
 '851',
 '上行/下行',
 '和平门——前门',
 '1171',
 '上行/下行',
 '前门——崇文门',
 '1634',
 '上行/下行',
 '崇文门——北京站',
 '1023',
 '上行/下行',
 '北京站——建国门',
 '

In [106]:
def get_station_distance(station_list):  #将站点上下行与对应距离组成字典以待使用
    station_distance = defaultdict(list)
    i = 0
    while(i <= len(station_list) - 3):
        station_distance[station_list[i]] = int(station_list[i+1]) 
        i = i + 3
    return station_distance  

In [107]:
station_distance = get_station_distance(station_list)

In [108]:
station_distance

defaultdict(list,
            {'苹果园——古城': 2606,
             '古城——八角游乐园': 1921,
             '八角游乐园——八宝山': 1953,
             '八宝山——玉泉路': 1479,
             '玉泉路——五棵松': 1810,
             '五棵松——万寿路': 1778,
             '万寿路——公主坟': 1313,
             '公主坟——军事博物馆': 1172,
             '军事博物馆——木樨地': 1166,
             '木樨地——南礼士路': 1291,
             '南礼士路——复兴门': 424,
             '复兴门——西单': 1590,
             '西单——天安门西': 1217,
             '天安门西——天安门东': 925,
             '天安门东——王府井': 852,
             '王府井——东单': 774,
             '东单——建国门': 1230,
             '建国门——永安里': 1377,
             '永安里——国贸': 790,
             '国贸——大望路': 1385,
             '大望路——四惠': 1673,
             '四惠——四惠东': 1715,
             '西直门——车公庄': 909,
             '车公庄——阜成门': 960,
             '阜成门——复兴门': 1832,
             '复兴门——长椿街': 1234,
             '长椿街——宣武门': 929,
             '宣武门——和平门': 851,
             '和平门——前门': 1171,
             '前门——崇文门': 1634,
             '崇文门——北京站': 1023,
             '北京站——建国门': 945

In [109]:
def search_1(graph,start,destination):
    pathes = [[start]]  # list 用来存储待搜索路径
    visited = set() # set用来存储已搜索的节点
    
    while pathes:
        path = pathes.pop(0)  #提取第一条路径
        froniter = path[-1]   #提取即将要探索的节点
        
        if froniter in visited: continue  #检查如果该点已经探索过 则不用再探索
            
        successsors = graph[froniter]
        
        for city in successsors:      #遍历子节点
            if city in path: continue  # check loop #检查会不会形成环
            
            new_path = path+[city]
            
            pathes.append(new_path)  #bfs     #将新路径加到list里面
            #pathes = [new_path] + pathes #dfs
            
            if city == destination:  #检查目的地是不是已经搜索到了
                return new_path
        visited.add(froniter)
        

In [110]:
search_1(station_connection,'古城','前门')

['古城',
 '八角游乐园',
 '八宝山',
 '玉泉路',
 '五棵松',
 '万寿路',
 '公主坟',
 '军事博物馆',
 '木樨地',
 '南礼士路',
 '复兴门',
 '西单',
 '宣武门',
 '和平门',
 '前门']

In [56]:
def search_2(graph,start,destination,search_strategy):
    pathes = [[start]]
    visited = set()# ！
    while pathes:
        path = pathes.pop(0)
        froniter = path[-1]
        if froniter in visited : continue# ！
            
        if froniter == destination:# ！
            return path# ！
        
        successsors = graph[froniter]
        
        for city in successsors:
            if city in path: continue  # check loop
            
            new_path = path+[city]
            
            pathes.append(new_path)  #bfs
            pathes = search_strategy(pathes)
            #sorted(pathes,key = lambda i:len(i),reverse=False)
        visited.add(froniter) # ！
       # if pathes and (destination == pathes[0][-1]):
       #     return pathes[0]  
        

In [59]:
def sort_by_lenth(pathes):
    #以换乘次数少进行排序，即路径最短
    return sorted(pathes,key = lambda i:len(i),reverse=False)

In [60]:
search_2(station_connection,'六里桥','前门',search_strategy=sort_by_lenth)

['六里桥', '六里桥东', '北京西站', '湾子', '达官营', '广安门内', '菜市口', '宣武门', '和平门', '前门']

In [83]:
station_distance

defaultdict(list,
            {'苹果园——古城': 2606,
             '古城——八角游乐园': 1921,
             '八角游乐园——八宝山': 1953,
             '八宝山——玉泉路': 1479,
             '玉泉路——五棵松': 1810,
             '五棵松——万寿路': 1778,
             '万寿路——公主坟': 1313,
             '公主坟——军事博物馆': 1172,
             '军事博物馆——木樨地': 1166,
             '木樨地——南礼士路': 1291,
             '南礼士路——复兴门': 424,
             '复兴门——西单': 1590,
             '西单——天安门西': 1217,
             '天安门西——天安门东': 925,
             '天安门东——王府井': 852,
             '王府井——东单': 774,
             '东单——建国门': 1230,
             '建国门——永安里': 1377,
             '永安里——国贸': 790,
             '国贸——大望路': 1385,
             '大望路——四惠': 1673,
             '四惠——四惠东': 1715,
             '西直门——车公庄': 909,
             '车公庄——阜成门': 960,
             '阜成门——复兴门': 1832,
             '复兴门——长椿街': 1234,
             '长椿街——宣武门': 929,
             '宣武门——和平门': 851,
             '和平门——前门': 1171,
             '前门——崇文门': 1634,
             '崇文门——北京站': 1023,
             '北京站——建国门': 945

In [95]:
def sort_by_distance(pathes): #使用之前得到的俩站间距离进行计算排序
    def get_distance_of_path(path):
        distance = 0
        for i in path[:-1]:  
            if(station_distance[i+'——'+path[path.index(i)+ 1]]):
                distance += station_distance[i+'——'+path[path.index(i)+ 1]]
            else:
                distance += station_distance[path[path.index(i)+ 1]+'——'+i]
        return distance
    return sorted(pathes,key=get_distance_of_path)

In [97]:
def get_distance_of_path(path):
        distance = 0
        for i in path[:-1]:  
            if(station_distance[i+'——'+path[path.index(i)+ 1]]):
                distance += station_distance[i+'——'+path[path.index(i)+ 1]]
            else:
                distance += station_distance[path[path.index(i)+ 1]+'——'+i]
        return distance

In [98]:
get_distance_of_path(['国家图书馆','白石桥南','白堆子'])

2039

In [96]:
search_2(station_connection,"六里桥","前门",search_strategy=sort_by_distance)

['六里桥', '六里桥东', '北京西站', '湾子', '达官营', '广安门内', '菜市口', '宣武门', '和平门', '前门']